This code implements a binary segmentation model following that described in the paper: [Deciphering impedance cytometry signals with neural networks](https://pubs.rsc.org/en/content/articlelanding/2022/lc/d2lc00028h).

This version is deprecated because I have migrated to a Jupyter notebook.

## Define model

In [2]:
import torch
import torch.nn as nn
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

# Set input size
input_size = 4

class BinarySegmentationRNN(nn.Module):
    def __init__(self, input_size):
        super(BinarySegmentationRNN, self).__init__()
        self.bi_lstm = nn.LSTM(input_size=input_size, hidden_size=512, batch_first=True, bidirectional=True)
        # Output size from bi-LSTM will be 512 * 2 due to bidirectionality
        # self.batch_norm = nn.BatchNorm1d(1024)  # Batch normalization for the LSTM output
        self.fc = nn.Linear(1024, 1)  # Maps to 1 output per timestep
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation to get probabilities for binary classification

    def forward(self, x):
        # x shape: (batch_size, sequence_length, 8)
        output, (hn, cn) = self.bi_lstm(x)  # output shape: (batch_size, sequence_length, 1024)

        # # Apply batch normalization
        # batch_size, seq_len, feature_size = output.size()
        # output = output.reshape(-1, feature_size)  # Reshape to (batch_size * sequence_length, 1024)
        # output = self.batch_norm(output)
        # output = output.view(batch_size, seq_len, feature_size)  # Reshape back to (batch_size, sequence_length, 1024)

        output = self.fc(output)  # (batch_size, sequence_length, 1)
        output = self.sigmoid(output)  # (batch_size, sequence_length, 1)
        return output.squeeze(-1)  # Optionally remove the last dimension for simplicity

# Load the segmentation model
save_path = os.path.join('models', 'binary_segmentation_rnn.pth')
segmentation_model = BinarySegmentationRNN(input_size=4).to(device)
segmentation_model.load_state_dict(torch.load(save_path))
segmentation_model.eval()


Device: cuda


BinarySegmentationRNN(
  (bi_lstm): LSTM(4, 512, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## Time-align two frequencies using the output of the segmentation model

In [ ]:
def load_signal_data(signal_data_loc, convert_to_numpy=True):
    signal_df = pd.read_csv(signal_data_loc)

    # Get a df that just has the signal timestamps and detrended values for each of the two signals
    ## First, narrow to just the columns t1 and t2 and all cols starting with 'Detrended1'
    signal_cols = signal_df.columns[signal_df.columns.str.contains('Detrended1')]
    signal1_cols = signal_cols[:4].tolist()
    signal2_cols = signal_cols[4:].tolist()
    # Now get signal1_df and signal2_df
    signal1_df = signal_df[['t1'] + signal1_cols].copy()
    signal2_df = signal_df[['t2'] + signal2_cols].copy()

    # t1 and t2 are timestamps. Rename to 'timestamp'
    signal1_df.rename(columns={'t1':'timestamp'}, inplace=True)
    signal2_df.rename(columns={'t2':'timestamp'}, inplace=True)

    # Set the timestamp as the index
    signal1_df.set_index('timestamp', inplace=True)
    signal2_df.set_index('timestamp', inplace=True)

    # Drop all rows of signal1_df and signal2_df that have indices higher than the min of the max index of the two dfs
    min_max_index = min(signal1_df.index.max(), signal2_df.index.max())
    signal1_df = signal1_df[signal1_df.index <= min_max_index]
    signal2_df = signal2_df[signal2_df.index <= min_max_index]

    # Merge the two dataframes as an outer join
    signal_df = pd.merge(signal1_df, signal2_df, on='timestamp', how='outer')

    # Linearly interpolate the missing values
    signal_df.interpolate(method='linear', inplace=True)

    if convert_to_numpy:
        # Convert to numpy array
        signal_data = signal_df.to_numpy()
    else:  
        signal_data = signal_df
        
    # Also get two 4-dimensional versions
    if convert_to_numpy:
        signal1_data = signal1_df.to_numpy()
        signal2_data = signal2_df.to_numpy()
    else:
        signal1_data = signal1_df
        signal2_data = signal2_df
    # And normalize
    signal1_data = (signal1_data - signal1_data.mean(axis=0)) / signal1_data.std(axis=0)
    signal2_data = (signal2_data - signal2_data.mean(axis=0)) / signal2_data.std(axis=0)

    return signal_data, signal1_data, signal2_data

In [8]:
import pandas as pd
import numpy as np

def find_time_bias(freq1: pd.Series, freq2: pd.Series, verbose=False) -> float:
    # Ensure the indices are floats
    freq1.index = freq1.index.astype(float)
    freq2.index = freq2.index.astype(float)

    # Merge the two series to ensure they have the same index
    freq1_aligned = pd.merge(pd.DataFrame(index=freq2.index), freq1, left_index=True, right_index=True, how='outer')
    freq2_aligned = pd.merge(pd.DataFrame(index=freq1.index), freq2, left_index=True, right_index=True, how='outer')

    # Interpolate missing values
    freq1_interpolated = freq1_aligned.interpolate()
    freq2_interpolated = freq2_aligned.interpolate()

    # Drop any remaining NaN values
    freq1_interpolated = freq1_interpolated.dropna()
    freq2_interpolated = freq2_interpolated.dropna()

    # Align the two series on the same time range
    start_time = max(freq1_interpolated.index.min(), freq2_interpolated.index.min())
    end_time = min(freq1_interpolated.index.max(), freq2_interpolated.index.max())

    freq1_interpolated = freq1_interpolated.loc[start_time:end_time]
    freq2_interpolated = freq2_interpolated.loc[start_time:end_time]

    # Compute cross-correlation to find the optimal lag
    freq1_normd = freq1_interpolated - freq1_interpolated.mean()
    freq2_normd = freq2_interpolated - freq2_interpolated.mean()
    freq1_normd = freq1_normd.squeeze()
    freq2_normd = freq2_normd.squeeze()

    correlation = np.correlate(freq1_normd, freq2_normd, mode='full')
    lag = correlation.argmax() - (len(freq1_interpolated) - 1)

    # Convert lag into time bias (in seconds)
    ## First, find the average time difference between two consecutive indices in freq1_interpolated
    time_diff = np.mean(np.diff(freq1_interpolated.index))
    time_bias = lag * time_diff

    time_bias

    return time_bias


In [9]:
def get_predicted_mask(signal_data, model):
    # Convert the sequence to a PyTorch tensor
    sequence_tensor = torch.tensor(signal_data.to_numpy(), dtype=torch.float32).unsqueeze(0).to(device)

    # Get the model prediction
    with torch.no_grad():
        model.eval()
        predicted_mask = model(sequence_tensor).squeeze().cpu().numpy()

    return predicted_mask

In [10]:
def align_freqs(signal1_data, signal2_data, model, start_time, end_time, verbose=False):
    # Add a predicted segmentation mask to the freq dfs
    predicted_mask1 = get_predicted_mask(signal1_data, model)
    predicted_mask2 = get_predicted_mask(signal2_data, model)

    # Make a series with index equal to each timestamp in the signal data and values equal to the predicted mask
    signal1_data['mask'] = predicted_mask1
    signal2_data['mask'] = predicted_mask2
    
    # Find the time bias to align the two signals
    bias = find_time_bias(signal1_data['mask'].loc[start_time:end_time], signal2_data['mask'].loc[start_time:end_time], verbose=verbose)

    # Correct the time index of signal2_data by adding time_bias to each index value of signal2_data
    signal2_data_corrected = signal2_data.copy()
    signal2_data_corrected.index = signal2_data_corrected.index + bias

    return signal1_data, signal2_data_corrected, bias

In [11]:
def align_freqs_and_get_predicted_mask(signal1_data, 
                                       signal2_data, 
                                       segmentation_model, 
                                       start_time, 
                                       end_time, 
                                       thresh=0.5,
                                       verbose=False):
    """
This function uses the `align_freqs` function to align the two signals
in time, and also uses the segmentation_model to get a predicted mask for
each of the two signal frequencies. Those two predicted masks are then combined,
as follows. The two masks are first averaged (for each time point), and then
are thresholded at `thresh` to get a final predicted mask. This final predicted mask
is binary.
    """
    ...

    return signal1_data, signal2_data_aligned, predicted_mask


In [12]:
# Try out the pipeline and plot results
# Select the dataset to use
set = '6'

# Set start and end times
start_time = 0.1
end_time = 10

# Load the signal data 
data_dir = os.path.join('/', 'home', 'cehrett', 'Projects', 'Microwave', 'data', 'labeled_segmentation_data')
signal_data_loc = os.path.join(data_dir, f'trended_and_detrended_signals_with_sigma_5_and_25_set{set}.csv')

signal_data, signal1_data, signal2_data = load_signal_data(signal_data_loc, convert_to_numpy=False)

# Plot the original data predicted masks
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()

# Add traces for each of freq1 and freq2
fig.add_trace(go.Scatter(x=signal1_data.index, y=get_predicted_mask(signal1_data, segmentation_model).squeeze(), mode='lines', name="Signal 1"))
fig.add_trace(go.Scatter(x=signal2_data.index, y=get_predicted_mask(signal2_data, segmentation_model).squeeze(), mode='lines', name="Signal 2"))

# Set plot layout
fig.update_layout(
    title="Predicted Masks Before Alignment",
    xaxis_title="Time",
    yaxis_title="Mask Value",
    legend_title="Legend"
)

# Show the plot
fig.show()

# Change the index of signal2_data, to be aligned with signal1_data
signal1_data, signal2_data, time_bias = align_freqs(signal1_data, signal2_data, segmentation_model, start_time, end_time)

# Plot the aligned signals
# Create a Plotly figure
fig = go.Figure()

# Add traces for each of freq1 and freq2
fig.add_trace(go.Scatter(x=signal1_data.index, y=signal1_data['mask'].squeeze(), mode='lines', name="Signal 1"))
fig.add_trace(go.Scatter(x=signal2_data.index, y=signal2_data['mask'].squeeze(), mode='lines', name="Signal 2"))

# Set plot layout
fig.update_layout(
    title="Aligned Signals",
    xaxis_title="Time",
    yaxis_title="Signal Value",
    legend_title="Legend"
)

# Print the time bias
print(f"Aligned the two frequencies over the time range [{start_time}, {end_time}]")
print(f"Time bias: {time_bias:.3f} seconds")

# Show the plot
fig.show()


Aligned the two frequencies over the time range [0.1, 10]
Time bias: 0.037 seconds


## Create classification CNN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# set kernel size
ks = (5,5)

# set height
h = 4

# set length
L = 20

class ClassificationCNN(nn.Module):
    def __init__(self, input_length=L, num_classes=8):
        super(ClassificationCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=20, kernel_size=ks, padding=2)
        self.bn1 = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=ks, padding=2)
        self.bn2 = nn.BatchNorm2d(20)
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=ks, padding=2)
        self.bn3 = nn.BatchNorm2d(20)
        
        # The output size should remain h x L due to padding, and there are 20 channels
        # Calculate the flattened size
        self.fc_input_size = 20 * h * input_length  # 20 channels, height h maintained, length 106 maintained
        self.fc = nn.Linear(self.fc_input_size, num_classes)
    
    def forward(self, x):
        # Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        # Block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        # Block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        # Flatten before passing to the fully connected layer
        x = torch.flatten(x, 1)

        # Fully connected layer
        x = self.fc(x)

        # Softmax for classification
        output = F.softmax(x, dim=1)
        return output

# Example usage:
model = ClassificationCNN()
input_tensor = torch.randn(1, 2, h, L)  # Example input batch where N=1, C=2, H=h, W=L
output = model(input_tensor)
print(output.shape)  # Should be torch.Size([1, 8]) indicating the output class probabilities for 8 classes


### Generate synthetic data to try out classification CNN

In [ ]:
# Create 8 different 8-dimensional means to represent our 8 classes
# First, create 8x8 array of all ones, scale it down, and then add a small amount to the diagonal
class_means = np.ones((8, 8)) * 0.8
class_means += np.eye(8) * 0.2

# Exponentiate each element of each mean, as a temperature control for differences between the means
raiseto_power = 1
class_means = np.exp(np.log(class_means) * raiseto_power)
class_means

In [ ]:
# Create a dataset. For each class, generate a sequence of n_events events with the corresponding class mean,
# create signals from those events, and create a target vector of classification labels.
n_samples = 100
seq_len = 1000
n_events = 5
impact_range = 15
noise_intensity = 0.3

# Generate event properties for all samples
event_positions = []
event_amplitudes = []
event_widths = []
class_labels = []
binary_masks = []
# Loop through the classes in class_means
for class_idx, class_mean in enumerate(class_means):
    for _ in range(n_samples):
        pos, amp, width = generate_event_properties(n_events, seq_len, class_mean, input_size=8)
        event_positions.append(pos)
        event_amplitudes.append(amp)
        event_widths.append(width)
        class_labels.append(class_idx)
        binary_masks.append(create_binary_mask(seq_len, pos, width))

# Generate the signals for all samples
signals = []
for pos, amp, width in zip(event_positions, event_amplitudes, event_widths):
    signals.append(create_signal(seq_len, pos, amp, width, impact_range, noise_intensity=noise_intensity, input_size=8))

In [ ]:
# Check if signals has any nan values
nan_values = np.isnan(signals)
print("Number of NaN values in signals:", np.sum(nan_values))

# Check how many non-nan values are in signals
non_nan_values = np.sum(~nan_values)
print("Number of non-NaN values in signals:", non_nan_values)

In [ ]:
# Take a look at a signal from each class in 4x2 subplot grid
fig, axs = plt.subplots(4, 2, figsize=(16, 16))
for i in range(8):
    ax = axs[i // 2, i % 2]
    signal_idx = class_labels.index(i)
    signal = signals[signal_idx]   
    ax.plot(signal)
    ax.set_title(f"Example of Class {i}")
    # Overlay the binary mask for the events
    mask = binary_masks[signal_idx]
    ax.plot(mask, linestyle="--", color="black")

plt.tight_layout()
plt.show()

In [ ]:
# Define a function which can take in the signals, the labels, and the binary masks, and 
# output both a tensor of shape (N, 2, h, L) where N is the total number of events in all the signals
# and also a tensor of shape (N,) containing the class labels
def create_classification_dataset(signals, class_labels, binary_masks, L):
    """
    Create a classification dataset from the signals, class labels, and binary masks.

    Parameters:
    signals (list of numpy.ndarray): A list of signals, each of shape (seq_len, 8).
    class_labels (list of int): A list of class labels corresponding to each signal.
    binary_masks (list of numpy.ndarray): A list of binary masks for each signal.

    Returns:
    tuple: A tuple containing:
        - input_tensor (torch.Tensor): A tensor of shape (N, 2, h, L) where N is the total number of events
                                       in all the signals.
        - target_tensor (torch.Tensor): A tensor of shape (N,) containing the class labels.
    """
    input_tensor = []
    target_tensor = []

    for signal, label, mask in zip(signals, class_labels, binary_masks):
        # For each contiguous segment of 1s in the binary mask, find the corresponding segment in the signal
        # and take the first four and last four dims of the signal to form a 2-channel input
        mask_segments = []
        signal_segments = []
        labels = []
        for i in range(len(mask)):
            if mask[i] == 1:
                if i == 0 or mask[i-1] == 0:
                    mask_segments.append([i])
                else:
                    mask_segments[-1].append(i)
        # For each mask segment, extract the corresponding segment from the signal
        for segment in mask_segments:
            start, end = segment[0], segment[-1]
            end = min(end + 1, start + L)
            signal_segment = signal[start:end]
            labels.append(label)

            # Pad the signal segments to length L (with 0s), so that each segment has shape Lx8
            padding = np.zeros((L - len(signal_segment), 8))
            padded_segment = np.concatenate([signal_segment, padding], axis=0)
            signal_segments.append(padded_segment)
            

        # Split each signal segment into two 4-channel segments of length L
        input_segments = []
        for segment in signal_segments:
            input_segment = np.stack([segment[:,:4], segment[:,4:]])
            input_segments.append(input_segment)

        # Append the input segments and labels to the tensors
        input_tensor.extend(input_segments)
        target_tensor.extend(labels)

    # Convert the lists to PyTorch tensors
    input_tensor = torch.tensor(input_tensor, dtype=torch.float32)
    target_tensor = torch.tensor(target_tensor, dtype=torch.long)

    return input_tensor, target_tensor

In [ ]:
# Create the synthetic classification dataset
input_tensor, target_tensor = create_classification_dataset(signals, class_labels, binary_masks, L)

# Print the shapes of the tensors
print("Input tensor shape:", input_tensor.shape)
print("Target tensor shape:", target_tensor.shape)

# Create a DataLoader to load the data in batches
from torch.utils.data import TensorDataset, DataLoader

# Define the batch size
batch_size = 128

# Create a TensorDataset
dataset = TensorDataset(input_tensor, target_tensor)

# Create a DataLoader
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the model, loss function, and optimizer
classification_model = ClassificationCNN(input_length=L, num_classes=8).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classification_model.parameters(), lr=0.001)

# Train the model
num_epochs = 30
for epoch in range(num_epochs):
    for input_batch, target_batch in data_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Move the input and target batch to the device
        input_batch = input_batch.to(device)
        target_batch = target_batch.to(device)

        # Forward pass
        output = classification_model(input_batch)

        # Calculate the loss
        loss = criterion(output, target_batch)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")



In [ ]:
# Define a function to calculate the accuracy of the model
def calculate_accuracy(y_true, y_pred):
    """
    Calculate the accuracy of the model.

    Parameters:
    y_true (numpy.ndarray): The ground truth labels.
    y_pred (numpy.ndarray): The predicted labels.

    Returns:
    float: The accuracy of the model.
    """
    correct = np.sum(y_true == y_pred)
    total = len(y_true)
    accuracy = correct / total
    return accuracy

# Get the model predictions
with torch.no_grad():
    classification_model.eval()
    predictions = []
    true_labels = []
    for input_batch, target_batch in data_loader:
        input_batch = input_batch.to(device)
        output = classification_model(input_batch)
        _, predicted = torch.max(output, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(target_batch.cpu().numpy())

# Calculate the accuracy of the model
accuracy = calculate_accuracy(np.array(true_labels), np.array(predictions))
print("Accuracy:", accuracy)

In [ ]:
#scr
pd.merge(signal1_data, signal2_data, on='timestamp', how='outer', suffixes=('_1', '_2')).interpolate(method='linear').dropna()

# Create a data ingestion pipeline which transforms data files into tensor datasets

In [ ]:
def convert_data_file_to_dataset(file_loc, segmentation_model, segmentation_threshold=0.5, device='cuda'):
    """
    Convert a data file to a dataset that can be used for classification.

    Parameters:
    file_loc (str): The location of the data file.
    segmentation_model (torch.nn.Module): The trained segmentation model.
    segmentation_threshold (float): The threshold for the segmentation model.
    device (str): The device to use for the tensors.

    Returns:
    tuple: A tuple containing:
        - input_tensor (torch.Tensor): A tensor of shape (N, 2, h, L) where N is the total number of events
                                       in all the signals.
        - target_tensor (torch.Tensor): A tensor of shape (N,) containing the class labels.
    """
    # Load the signal data
    signal_data, signal1_data, signal2_data = load_signal_data(file_loc, convert_to_numpy=False)

    # Align the two signals
    start_time = 0.1
    end_time = 10
    signal1_data, signal2_data, time_bias = align_freqs(signal1_data, signal2_data, segmentation_model, start_time, end_time)

    # Join the two signals into a single dataframe
    signal_data = pd.merge(signal1_data, signal2_data, on='timestamp', how='outer', suffixes=('_1', '_2'))

    # Linearly interpolate the missing values
    signal_data.interpolate(method='linear', inplace=True)

    # Drop any remaining NaN values
    signal_data.dropna(inplace=True)

    # Separate the 'mask_1' and 'mask_2' columns from the signal data, and drop them from the original df
    mask1_data = signal_data['mask_1']
    mask2_data = signal_data['mask_2']
    signal_data.drop(columns=['mask_1', 'mask_2'], inplace=True)

    # Threshold each of the two masks
    mask1_data = (mask1_data > segmentation_threshold)
    mask2_data = (mask2_data > segmentation_threshold)

    # Get a single mask which is the logical disjunction of the two mask series
    mask_data = mask1_data | mask2_data
    mask_data = mask_data.astype(int)

    # Create the classification dataset
    input_tensor, target_tensor = create_classification_dataset([signal_data.to_numpy()], [0], [mask_data.to_numpy()], L)

    return input_tensor, target_tensor

In [ ]:
# Make a dataset

# Select the dataset to use
set = '6'

# Load the signal data and put it in a numpy array of shape (L, 8) where L is the length of the sequence
data_dir = os.path.join('/', 'home', 'cehrett', 'Projects', 'Microwave', 'data', 'labeled_segmentation_data')
signal_data_loc = os.path.join(data_dir, f'trended_and_detrended_signals_with_sigma_5_and_25_set{set}.csv')

input_tensor, target_tensor = convert_data_file_to_dataset(signal_data_loc, segmentation_model)